##### ═══════════════════════════════════════════════════════════
##### Paid Media Summary - Part 1 of 2 - Campaign-level analysis
##### ═══════════════════════════════════════════════════════════

In [1]:
import pandas as pd
import numpy as np

# Load pseudonymized data with proper dtype for project_no
s_df = pd.read_csv('streaming_pseudonymized_new.csv.gz', compression='gzip', dtype={'project_no': 'Int64'})
m_df = pd.read_csv('marketing_pseudonymized_new.csv', dtype={'project_no': 'Int64'}, low_memory=False)
print(f"Streaming: {s_df.shape[0]:,} rows")
print(f"Marketing: {m_df.shape[0]:,} rows")
print(f"Marketing campaigns: {m_df['campaign_id'].nunique()}")


# Convert dates
s_df['report_date'] = pd.to_datetime(s_df['report_date'])
m_df['campaign_start_date'] = pd.to_datetime(m_df['campaign_start_date'])
m_df['campaign_end_date'] = pd.to_datetime(m_df['campaign_end_date'])
m_df['marketing_report_date'] = pd.to_datetime(m_df['marketing_report_date'])
print("✓ Dates converted and project numbers loaded as integers")

Streaming: 614,005 rows
Marketing: 13,835 rows
Marketing campaigns: 111
✓ Dates converted and project numbers loaded as integers


In [2]:
# Marketing and streaming cutoffs
Q1_START = pd.to_datetime('2023-01-01')
STREAMING_DATA_END = pd.to_datetime('2023-03-30')
MARKETING_DATA_END = pd.to_datetime('2023-03-31')  # One day later to capture end-of-month reports

# Keep only marketing reports up to March 31
m_df_filtered = m_df[m_df['marketing_report_date'] <= MARKETING_DATA_END].copy()
print(f"Daily reports up to {MARKETING_DATA_END.date()}: {len(m_df_filtered):,} (from {len(m_df):,} total rows)")
print(f"Excluded reports after cutoff: {len(m_df) - len(m_df_filtered):,}")

Daily reports up to 2023-03-31: 11,273 (from 13,835 total rows)
Excluded reports after cutoff: 2,562


In [3]:
# Part 1: Marketing metrics by campaign
campaign_metrics = m_df_filtered.groupby(['campaign_id', 'platform_category', 'canonical_artist', 'canonical_product']).agg({
    'objective': 'first',
    'project_no': 'first',
    'product_number_type': 'first',
    'campaign_start_date': lambda x: pd.to_datetime(x).min(),
    'marketing_report_date': 'max',  # to get last report date
    'spend': 'sum',
    'impressions': 'sum',
    'reach': 'max',
    'video_views': 'sum',
    'link_clicks': 'sum',
    'interactions': 'sum',
    'comments': 'sum',
    'likes': 'sum',
    'reactions': 'sum',
    'shares': 'sum',
    'follows': 'sum',
    'conversions': 'sum',
    'all_conversions': 'sum'
}).reset_index()

# Take the higher of the two conversion columns
campaign_metrics['conversions'] = campaign_metrics[['conversions', 'all_conversions']].max(axis=1)
campaign_metrics.drop('all_conversions', axis=1, inplace=True)

# Use the last marketing report date as campaign end date
campaign_metrics['campaign_end_date'] = campaign_metrics['marketing_report_date']
campaign_metrics.drop('marketing_report_date', axis=1, inplace=True)

campaign_metrics['campaign_duration_days'] = (
    pd.to_datetime(campaign_metrics['campaign_end_date']) - 
    pd.to_datetime(campaign_metrics['campaign_start_date'])
).dt.days + 1

# Create composite metrics
def calculate_composite_metrics(row):
    platform = row['platform_category'].lower()
    
    # Awareness (avoid double-counting impressions/reach)
    awareness_base = max(row['impressions'], row['reach'])
    row['total_awareness_metrics'] = awareness_base + row['video_views']
    
    # Traffic
    row['total_traffic_metrics'] = row['link_clicks'] + row['conversions']
    
    # Engagement (platform-specific)
    if platform in ['facebook', 'instagram']:
        row['total_engagement_metrics'] = row['interactions']
    elif platform in ['google', 'youtube']:
        row['total_engagement_metrics'] = max(0, row['interactions'] - row['video_views'])
    elif platform == 'tiktok':
        row['total_engagement_metrics'] = (
            row['comments'] + row['likes'] + row['reactions'] +
            row['shares'] + row['follows']
        )
    else:  # snapchat
        row['total_engagement_metrics'] = 0
    
    return row

campaign_metrics = campaign_metrics.apply(calculate_composite_metrics, axis=1)

print(f"✓ Campaign metrics aggregated: {len(campaign_metrics):,} campaign×platform×artist×product combinations")

✓ Campaign metrics aggregated: 271 campaign×platform×artist×product combinations


In [4]:
# Part 2: For each campaign, calculate streaming metrics
s_df['report_date'] = pd.to_datetime(s_df['report_date'])
campaign_metrics['campaign_start_date'] = pd.to_datetime(campaign_metrics['campaign_start_date'])
campaign_metrics['campaign_end_date'] = pd.to_datetime(campaign_metrics['campaign_end_date'])


def get_streaming_metrics(row):
    """Calculate streaming metrics - only counts artist streams if the promoted product exists in streaming data"""
    artist_streams = s_df[s_df['project_no'] == row['project_no']].copy()
    
    # Check if promoted product EXISTS in streaming data at all 
    #(if not, any other streams of artist catalog cannot plausibly be attributed to the campaign)
    promoted_product_exists = artist_streams[
    (artist_streams['canonical_product'] == row['canonical_product'])&
    (artist_streams['report_date'] >= row['campaign_start_date']) &
    (artist_streams['report_date'] <= row['campaign_end_date'] + pd.Timedelta(days=10))
]['streams'].sum()
    
    # If promoted product never appears in streaming data, zero out everything
    if not promoted_product_exists:
        return pd.Series({
            'streams_during_campaign': 0,
            'streams_10day_post': 0,
            'baseline_streams_14day_pre': 0
        })

    # If promoted product WAS streamed, count all artist streams (catalog halo effect)
    
    # During campaign
    during = artist_streams[
        (artist_streams['report_date'] >= row['campaign_start_date']) &
        (artist_streams['report_date'] <= row['campaign_end_date'])
    ]['streams'].sum()
    
    # 10 days post
    post = artist_streams[
        (artist_streams['report_date'] > row['campaign_end_date']) &
        (artist_streams['report_date'] <= row['campaign_end_date'] + pd.Timedelta(days=10))
    ]['streams'].sum()
    
    # 14 days pre (baseline)
    pre = artist_streams[
        (artist_streams['report_date'] >= row['campaign_start_date'] - pd.Timedelta(days=14)) &
        (artist_streams['report_date'] < row['campaign_start_date'])
    ]['streams'].sum()
    
    baseline = pre / 14 if pre > 0 else 0
    
    return pd.Series({
        'streams_during_campaign': during,
        'streams_10day_post': post,
        'baseline_streams_14day_pre': baseline
    })

print("Calculating streaming metrics for each campaign...")
streaming_metrics = campaign_metrics.apply(get_streaming_metrics, axis=1)
campaign_summary = pd.concat([campaign_metrics, streaming_metrics], axis=1)
print(f"✓ Streaming metrics added")

Calculating streaming metrics for each campaign...
✓ Streaming metrics added


In [5]:
# Create streaming trackability flag
campaign_summary['streaming_trackable'] = campaign_summary['product_number_type'].apply(
    lambda x: 'track' if x == 'Track URI' else 'non_track'
)

print(f"\nStreaming Trackability:")
print(campaign_summary['streaming_trackable'].value_counts())
print(f"Track campaigns: {(campaign_summary['streaming_trackable'] == 'track').sum()}")
print(f"Non-track campaigns: {(campaign_summary['streaming_trackable'] == 'non_track').sum()}")



Streaming Trackability:
streaming_trackable
track        139
non_track    132
Name: count, dtype: int64
Track campaigns: 139
Non-track campaigns: 132


In [6]:
# Phase windows (explicit, non-overlapping)
campaign_summary['baseline_start'] = campaign_summary['campaign_start_date'] - pd.Timedelta(days=14)
campaign_summary['baseline_end']   = campaign_summary['campaign_start_date'] - pd.Timedelta(days=1)

campaign_summary['during_start'] = campaign_summary['campaign_start_date']
campaign_summary['during_end']   = campaign_summary['campaign_end_date']

campaign_summary['decay_start'] = campaign_summary['campaign_end_date'] + pd.Timedelta(days=1)
campaign_summary['decay_end']   = campaign_summary['campaign_end_date'] + pd.Timedelta(days=10)


In [7]:
def overlap_days(a_start, a_end, b_start, b_end):
    start = max(a_start, b_start)
    end   = min(a_end, b_end)
    return max(0, (end - start).days + 1)

def compute_phase_overlaps(row):
    same_artist = campaign_summary[
        (campaign_summary['project_no'] == row['project_no']) &
        (
            (campaign_summary['campaign_id'] != row['campaign_id']) |
            (campaign_summary['platform_category'] != row['platform_category'])
        )
    ]

    baseline_days = during_days = decay_days = 0

    for _, other in same_artist.iterrows():
        baseline_days += overlap_days(
            row['baseline_start'], row['baseline_end'],
            other['campaign_start_date'], other['campaign_end_date']
        )
        during_days += overlap_days(
            row['during_start'], row['during_end'],
            other['campaign_start_date'], other['campaign_end_date']
        )
        decay_days += overlap_days(
            row['decay_start'], row['decay_end'],
            other['campaign_start_date'], other['campaign_end_date']
        )

    return pd.Series({
        'overlap_days_baseline': baseline_days,
        'overlap_days_during': during_days,
        'overlap_days_decay': decay_days,
        'overlap_ratio_baseline': baseline_days / 14,
        'overlap_ratio_during': during_days / row['campaign_duration_days'],
        'overlap_ratio_decay': decay_days / 10
    })

campaign_summary = campaign_summary.join(
    campaign_summary.apply(compute_phase_overlaps, axis=1)
)


In [8]:
# Check baseline situation in the dataset
print("Baseline Analysis:")
print(f"\nTotal campaigns: {len(campaign_summary)}")
print(f"Campaigns with baseline > 0: {(campaign_summary['baseline_streams_14day_pre'] > 0).sum()}")
print(f"Campaigns with baseline = 0: {(campaign_summary['baseline_streams_14day_pre'] == 0).sum()}")

print(f"\nPercentage with baseline: {(campaign_summary['baseline_streams_14day_pre'] > 0).sum() / len(campaign_summary) * 100:.1f}%")

# See distribution
print("\nBaseline value distribution:")
print(campaign_summary['baseline_streams_14day_pre'].describe())

# Show examples of campaigns without baseline
print("\nSample campaigns with NO baseline (baseline = 0):")
no_baseline = campaign_summary[campaign_summary['baseline_streams_14day_pre'] == 0]
print(no_baseline[['campaign_id', 'platform_category', 'canonical_artist', 'canonical_product', 
                    'campaign_start_date', 'streams_during_campaign']].head(10))

print("\nBaseline overlap check:")
print(f"Campaigns with baseline overlap days > 0: {(campaign_summary['overlap_days_baseline'] > 0).sum()}")
print(f"Percentage with baseline overlap: {(campaign_summary['overlap_days_baseline'] > 0).sum() / len(campaign_summary) * 100:.1f}%")

print("\nBaseline overlap (days) distribution:")
print(campaign_summary['overlap_days_baseline'].describe())

print("\nBaseline vs baseline-overlap correlation:")
print(campaign_summary[['baseline_streams_14day_pre', 'overlap_days_baseline']].corr())



Baseline Analysis:

Total campaigns: 271
Campaigns with baseline > 0: 164
Campaigns with baseline = 0: 107

Percentage with baseline: 60.5%

Baseline value distribution:
count       271.000000
mean      26807.791249
std       82582.672438
min           0.000000
25%           0.000000
50%        4877.357143
75%       17514.142857
max      715595.785714
Name: baseline_streams_14day_pre, dtype: float64

Sample campaigns with NO baseline (baseline = 0):
     campaign_id platform_category canonical_artist   canonical_product  \
2    24210616442          snapchat   Gurney_Halleck  Packards_Vibration   
7   120285243753          snapchat   Gurney_Halleck  Packards_Vibration   
9   130740435072           youtube  One_Adam_Kesher    White_Tail_Falls   
14  157858920231          facebook      Bill_Dayton      Angel_Vanishes   
20  157858920231          facebook    Juana_Durango     Without_a_Doubt   
22  157858920231          facebook          Mr_Eddy  Do_It_for_Van_Gogh   
23  157858920231     

In [9]:
def count_concurrent_campaigns(row):
    """Count how many OTHER campaigns (different campaign_id or platform_category)
    for this artist overlapped"""

    same_artist = campaign_summary[
    (campaign_summary['project_no'] == row['project_no']) &
    (
        (campaign_summary['campaign_id'] != row['campaign_id']) |
        (campaign_summary['platform_category'] != row['platform_category'])
    )
]
    
    overlaps = same_artist[
        (same_artist['campaign_start_date'] <= row['campaign_end_date']) &
        (same_artist['campaign_end_date'] >= row['campaign_start_date'])
    ]
    
    return len(overlaps)
    
campaign_summary['any_overlap_count'] = campaign_summary.apply(
    count_concurrent_campaigns, axis=1
)


In [10]:
# Save campaign summary for paid media analysis
campaign_summary.to_csv('final_campaign_summary_paid_media.csv', index=False)

In [11]:
print("="*70)
print("DISTRIBUTION ANALYSIS - Mean vs Median")
print("="*70)

# Metrics to analyze
metrics_to_check = ['campaign_duration_days', 'spend', 'any_overlap_count']

for metric in metrics_to_check:
    print(f"\n{metric.upper()}:")
    print(f"  Mean:   {campaign_summary[metric].mean():.0f}")
    print(f"  Median: {campaign_summary[metric].median():.0f}")
    print(f"  Std:    {campaign_summary[metric].std():.0f}")
    print(f"  Min:    {campaign_summary[metric].min():.0f}")
    print(f"  Max:    {campaign_summary[metric].max():.0f}")
    print(f"  25th percentile: {campaign_summary[metric].quantile(0.25):.0f}")
    print(f"  75th percentile: {campaign_summary[metric].quantile(0.75):.0f}")
    
    # Check skewness
    skew = campaign_summary[metric].skew()
    print(f"  Skewness: {skew:.0f} ", end="")

print("\n" + "="*70)
print("SPEND DISTRIBUTION BY PLATFORM:")
print("="*70)
for platform in sorted(campaign_summary['platform_category'].unique()):
    platform_data = campaign_summary[campaign_summary['platform_category'] == platform]['spend']
    campaign_count = len(platform_data)
    print(f"\n{platform} ({campaign_count} campaigns):")
    print(f"  Mean: ${platform_data.mean():,.0f}  |  Median: ${platform_data.median():,.0f}")
    print(f"  Range: ${platform_data.min():,.0f} - ${platform_data.max():,.0f}")

DISTRIBUTION ANALYSIS - Mean vs Median

CAMPAIGN_DURATION_DAYS:
  Mean:   19
  Median: 13
  Std:    17
  Min:    1
  Max:    82
  25th percentile: 8
  75th percentile: 22
  Skewness: 2 
SPEND:
  Mean:   831
  Median: 450
  Std:    1330
  Min:    0
  Max:    9421
  25th percentile: 281
  75th percentile: 843
  Skewness: 4 
ANY_OVERLAP_COUNT:
  Mean:   3
  Median: 2
  Std:    2
  Min:    0
  Max:    8
  25th percentile: 1
  75th percentile: 4
  Skewness: 1 
SPEND DISTRIBUTION BY PLATFORM:

facebook (25 campaigns):
  Mean: $1,307  |  Median: $350
  Range: $69 - $8,643

google (4 campaigns):
  Mean: $446  |  Median: $238
  Range: $27 - $1,281

instagram (82 campaigns):
  Mean: $1,073  |  Median: $617
  Range: $3 - $8,928

other (11 campaigns):
  Mean: $0  |  Median: $0
  Range: $0 - $0

snapchat (11 campaigns):
  Mean: $721  |  Median: $562
  Range: $281 - $2,469

tiktok (48 campaigns):
  Mean: $1,114  |  Median: $617
  Range: $130 - $9,421

youtube (90 campaigns):
  Mean: $460  |  Median:

**Distribution rationale**: Our campaign distribution is skewed (max 82 days vs 75th percentile at 22 days), so we use median instead of mean. The same applies to spend, where high-budget outliers (over 9k) pull the mean above the 75th percentile.

**Concurrency implications**: Medium-high campaign concurrency (median: 2 concurrent campaigns) means streaming spikes during campaign windows have confounding factors, making correlation claims not easily defensible.

**Platform spending patterns**: Spend distribution reveals two strategic approaches:

1) High-variance platforms (indicative of experimentation): Instagram, Facebook, TikTok and
2) Consistent-spend platforms (indicative of a standardized approach): YouTube, Snapchat, Google
   
**Key insight: Instagram and YouTube dominate campaign volume (82 and 90 campaigns respectively), but with opposite budget approaches — Instagram varies widely, YouTube stays mostly lean.**